# Demystifying table sharding, tablets, and data distribution
As a distributed SQL database, YugabyteDB stores data differently than a stand-alone, monolithic  database. Because data is stored differently, YugabyteDB reads data differently as well.

Like a monolithic relational databases, YugabyteDB stores data in tables. However, in YugabyteDB, a table consists of tablets. 

A tablet represents a table shard which contains a set of rows for the logical table. Each tablet is a customized RocksDB instance. A tablet leader has a peer group known as tablet followers, and this group of tablet peers exists as a Raft consensus group. YugabyteDB calls this distributed document store, DocDB.

In this lab, using Explain Plans, built-in functions, and custom utilities for YB-TServer metrics, you will learn about how YugabyteDB stores data for a table, as table shards, known as tablets. You will also learn how YugabyteDB reads tablet data during query execution.


## Requirements
Before running the cells in this notebook, you must first edit and execute all the cells in the first notebook:
- `01_Lab_Setup.ipynb`

## Connect to `db_ybu` using the PostgreSQL driver for Python
Run all the cells in this section:
- Connect using Python and PostgreSQL driver
- Load the SQL magic extension
- Create prepared statements for using utility metrics


In [1]:
# connect use Python 3.7.9+
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
# Use %store -r to read 01_Lab_Requirements_Setup variables

%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02YB_PATH=${1}
%store -r MY_HOST_IPv4_03
%store -r MY_GITPOD_WORKSPACE_URL

db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME


connection_str='postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name

# engine = create_engine(connection_str)

no stored variable or alias MY_HOST_IPv4_02YB_PATH=$1


In [2]:
%reload_ext sql

# SQL magic for python connection string
%sql {connection_str}

### Create prepared utility statements

> IMPORTANT!
>   
> In order to create the prepared statements for the SQL magic connection, you must run the following cell!!!
> 
> Do not skip this step.
> 

In [3]:
#%% python, but prepared statements as sql magic
if (MY_GITPOD_WORKSPACE_URL is None):
    a = %sql select fn_yb_create_stmts()
else:
    WORKSPACE_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
    a = %sql select fn_yb_create_stmts(:WORKSPACE_URL)

print (a)

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.
+----------------------------------+
|        fn_yb_create_stmts        |
+----------------------------------+
| 2022-07-26 11:38:09.453429-07:00 |
+----------------------------------+


Confirm that the following query returns a count of 3 (for three prepared statements).

In [4]:
%%sql 
select count(*) from pg_prepared_statements where 1=1 and name in ('stmt_util_metrics_snap_tablet','stmt_util_metrics_snap_table','stmt_util_metrics_snap_reset')

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.


count
3


## q1 | Create a table with no Primary Key (PK)
For this first example, you will create a table without explicitly defining a primary key. You will then review an Explain Plan for a query of the table as well as query metrics.

In [ ]:
%%sql /* create table no PK and insert rows */
drop table if exists tbl_no_pk;

create table if not exists tbl_no_pk (k int, v text);

insert into tbl_no_pk (k,v)
select g.id, format('%s%s',chr(97+CAST(random() * 25 AS INTEGER)),chr(97+CAST(random() * 25 AS INTEGER)))
from generate_series(1, 1000) AS g (id);

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -c "\d+ tbl_no_pk"

### q1a | Query the table for a value of k

The following is a query that will show the Explain Plan for the query. The first statement resets the metrics that will be captured after running the query. The query itself has a `where` clause predicate for `k` . 

In [ ]:
%%sql /* explain plan */
execute stmt_util_metrics_snap_reset;
explain (analyze, costs off, verbose, timing on) 
select '' _
    , k
    , v
from tbl_no_pk 
where 1=1 
and  k=1000
-- limit 30
;

#### q1a | Explain Plan (above ^^)
Because YugabyteDB reuses the PostgreSQL query layer, it is possible to view an Explain Plan for a query. A query optimizer generates the Explain Plan for query execution.
The Explain Plan for this query shows:
- `Seq Scan on public.tbl_no_pk (actual time=7.912..11.720 rows=1 loops=1)`

A `Seq Scan` is a full table scan, meaning that potentially all the rows in the table will be read.

- `Rows Removed by Filter: 999`

999 rows were removed, leaving the remain 1 row.

The Explain Plan infers how the query accesses tablet data, but does **NOT** show the metrics for tablet access.Using a custom utility, you can view the metrics for the query execution in the notebook cell below.

In [ ]:
%%sql /* metrics */
execute stmt_util_metrics_snap_table;

#### q1a | Metrics (above ^^)

The Explain Plan revealed that the query read 1000 rows, and filter out 999 rows, to return a 1 row result.

The Metrics reports shows that the query access 3 tablets for the table (`tbl_no_pk`). Each tablet has 1 seek of an offset, and then per offset, 900+ reads.

Here is how to read the output of the Metrics report:
- row_name `db_ybu | tbl_no_pk | https://7000-yugabytedbu-ysqlpe03whe-um13wtg7p0o.ws-us54.gitpod.io/table?id=00004000000030008000000000004039 | bc64657cf03842fab00a416cc67c8413 | L`
  - database --> `db_ybu`
  - table or index or materialized view  --> `tbl_no_pk`
  - table_id --> `https://7000-yugabytedbu-ysqlpe03whe-um13wtg7p0o.ws-us54.gitpod.io/table?id=00004000000030008000000000004039` --> url to browser -->  table_id? --> `00004000000030008000000000004039`
  - tablet_id --> `4aed1608c05d40edb7105b11abc629cd`
  - isLeader --> `L`
- rocksdb_number_db_seek
  - The number of Seek() RocksDB API calls, which means the number of seeks for offsets 
- rocksdb_number_db_next
  - The number of Next() RocksDB API calls, which means the number of reads from the offset
- rows_inserted
  - number of rows inserted 

### View the Table details in the YB-Master web ui
To better understand the Metrics report, you can view the details of a table in the YB-Master web ui.

From the `row_name` column, copy the Gitpod URL (`https://7000-yugabytedbu-ysqlpe03whe-um13wtg7p0o.ws-us54.gitpod.io/table?id=00004000000030008000000000004039)` into your web browser, and view the table details, namely the Columns section and the Tablets section

> Note
>  
>  In gitpod, only the `127.0.0.1` host is available on port 7000, and will show as `https://7000-yugabytedbu-ysqlpe03whe-um13wtg7p0o.ws-us54.gitpod.io/table?id=00004000000030008000000000004039` . You are not able to browse `127.0.0.2` or `127.0.0.3`

Or, you can run code below to get the URL to open in your web browser.

In [36]:
#%% python, but prepared statements as sql magic
WORKSPACE_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
view_url = %sql select fn_get_table_id(:WORKSPACE_URL,8200,'tbl_no_pk') as open_this_url ;

print (view_url)

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
0 rows affected.
+---------------+
| open_this_url |
+---------------+
+---------------+


#### Review the Column section
The column section shows details about each column in the table, index, or materialized view. In the column section for `tbl_no_pk`, you will see:


| Column | ID	| Type |
|--------|------|------|
| ybrowid    | 0	| binary NOT NULL PARTITION KEY | 
| k	         | 1	| int32 NULLABLE NOT A PARTITION KEY | 
| v	         | 2	| string NULLABLE NOT A PARTITION KEY | 

<br/>

> Important!
>  
> When you create a table without a Primary Key (PK), YugabyteDB creates a hidden primary key for you, `ybrowid`. You will see this hidden PK in the YB-Master web ui for the `tbl_no_pk` table.


`ybrowid` functions as a hidden PK for a row with a random Universal Unique Identifier (UUID) value. Using  `\d` or `\d+` will not show the `ybrowid` primary key. It is not possible to query the `ybrowid` value.

In the case of the `tbl_no_pk` table,  `ybrowid` is a hidden primary key that functions as the partition key.

##### Partition Key
When YugabyteDB processes an insert query for a row of data into a table with a consistent hash sharding strategy, YugabyteDB uses the shard key (shown as `PARTITION KEY`) to distribute the data among the tablet leaders. 

With consistent hash sharding, a partitioning algorithm distributes data evenly and randomly across shards. By computing a consistent hash on the partition key (or keys) of a given row, YugabyteDB knows where to insert the row among the tablet leaders.



#### Review the Tablet section
For the given table, the Tablet section shows the details for the existing tablets. Of particular interest are the number of tablet leaders and the partition strategy. Here is an abridged example of the Tablet section for the `tbl_no_pk` table:

| Tablet ID |	Partition	| SplitDepth	| State	| Hidden	| Message	| RaftConfig|
|--|--|--|--|--|--|--|
| some_uuid_1 |	`hash_split: [0x5555, 0xAAAA)` |	0	| Running|	false| Tablet reported with an active leader	|<li>FOLLOWER: 127.0.0.1 <li>FOLLOWER: 127.0.0.3<li>LEADER: 127.0.0.2  |
| some_uuid_2	| `hash_split: [0xAAAA, 0xFFFF)`	| 0 |  Running |false |	Tablet reported with an active leader |<li>FOLLOWER: 127.0.0.1 <li>LEADER: 127.0.0.3 <li>FOLLOWER: 127.0.0.2 |
| some_uuid_3 <br>(tablet leader where the row lives) |	`hash_split: [0x0000, 0x5555)` |	0 |	Running | 	false	| Tablet reported with an active leader |	<li>LEADER: 127.0.0.1<li>FOLLOWER: 127.0.0.3<li>FOLLOWER: 127.0.0.2 |

##### Why are there three tablets each with a tablet_id ?

Various properties and configurations determine the number of tablets for a specific table in a given YugabyteDB cluster. 

Your YugabyteDB cluster is a three node cluster. Each node runs two processes: a YB-Master process and a YB-TServer process. Your cluster has a replication factor of three, which means that each tablet leader will have two tablet followers. 

Certain flags specify the number of shards per YB-TServer. Your cluster has two specific flags: 
-  `yb_num_shards_per_tserver=1`
-  `ysql_num_shards_per_tserver=1`

These flags dictate that each YB-TServer process will initially create just one tablet leader for a given table with hash sharding.

For this reason, with your three node cluster, a table will have a single tablet leader per node.  And since there are three nodes in your YugabyteDB cluster, there will be a total of three tablet leaders for a given table with hash sharding. The replication factor of three specifies that each tablet leader will have two tablet followers. 

##### What is the `hash_split` value in the Partition column? 

For a table with hash sharding, each tablet has a Partition value that shows a  `hash_split` . The `hash_split` details a range of hexadecimal values. 
```
hash_split: [0x0000, 0x5555)
hash_split: [0x5555, 0xAAAA)
hash_split: [0xAAAA, 0xFFFF)
```
The conversion of the above hexadecimal values to integer values is as follows:
```
0-21845
21845-43690
43690-65535
```

In other words, the Partition value for hash sharding represents a range of hexadecimal values starting with 0 and ending with 65535. Here is illustration of consistent hash sharding:

<img  style="background-color:#FFF" src="assets/hash_sharding.png" >

The primary key is the partition key. The hash value of the partition key results in a hexadecimal value. This value falls between the lower and upper values of a `hash_split` range for a tablet leader. 

YSQL has an built-in function that shows the integer form of the hash code: `yb_hash_code()` . This built-in function returns an integer, making it easier to interpret the hexadecimal values of a hash_splt range.

However, because YugabyteDB does not expose `ybrowid`, it is not possible to use `yb_hash_code()` to determine what is the integer valie for the hash code of `ybrowid`. 

## q2 | Alter the table and add a Primary Key
To help illuminate how YugabyteDB distributes data, you can create primary key the `k` column of the table. Later, you will use the `yb_code_hash()` built-in function and the primary key to show where various rows live on a given tablet.

Because there is not an existing primary key (PK) for `tbl_no_pk`, it is possible to add a one.

> IMPORTANT
> Once you create a primary key on a table, you can not alter it.

 - `PRIMARY KEY (k)` for hash  [DEFAULT]
 - `PRIMARY KEY (k hash)` for hash
 - `PRIMARY KEY (k asc)` for range

> NOTE
> 
> In this scenario, when you add a primary key to a table that does not have one, YugabyteDB will need to recreate the entire table behind the scenes and distribute the data according to PK sharding algorithm, range or hash. The previous table with the table id will show the table name as table_name_temp_old. The newly created table will have a new table id.

In [ ]:
%%sql /* alter table rename AND add PK */

drop table if exists tbl_pk_hash;
-- rename table
alter table if exists tbl_no_pk rename to tbl_pk_hash;
-- add PK as hash
alter table tbl_pk_hash add primary key (k hash);

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" #\d+
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -c "\d+ tbl_pk_hash"

### View the Table details in the YB-Master web ui
In your web browser, open the following URL.

In [ ]:
#%% python, but prepared statements as sql magic
WORKSPACE_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
view_url = %sql select fn_get_table_id(:WORKSPACE_URL,8200,'tbl_pk_hash') as open_this_url ;

print (view_url)

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
0 rows affected.
+---------------+
| open_this_url |
+---------------+
+---------------+


#### Review the Column section
The column section shows details about each column in the table, index, or materialized view. In the column section for `tbl_pk_hash`, you will see:


| Column | ID	| Type |
|--------|------|------|
| k	         | 1	| int32 PARTITION KEY | 
| v	         | 2	| string NULLABLE NOT A PARTITION KEY | 

<br/>

Because there is a primary key defined for this table, there is no `ybrowid` column.

#### Review the Tablet section
The Tablet section shows the details for the existing tablets. Here is an abridged example of the Tablet section for the `tbl_pk_hash` table:

| Tablet ID |	Partition	| SplitDepth	| State	| Hidden	| Message	| RaftConfig|
|--|--|--|--|--|--|--|
| some_uuid_1 |	`hash_split: [0x5555, 0xAAAA)` |	0	| Running|	false| Tablet reported with an active leader	|<li>FOLLOWER: 127.0.0.1 <li>FOLLOWER: 127.0.0.3<li>LEADER: 127.0.0.2  |
| some_uuid_2	| `hash_split: [0xAAAA, 0xFFFF)`	| 0 |  Running |false |	Tablet reported with an active leader |<li>FOLLOWER: 127.0.0.1 <li>LEADER: 127.0.0.3 <li>FOLLOWER: 127.0.0.2 |
| some_uuid_3 <br>(tablet leader where the row lives) |	`hash_split: [0x0000, 0x5555)` |	0 |	Running | 	false	| Tablet reported with an active leader |	<li>LEADER: 127.0.0.1<li>FOLLOWER: 127.0.0.3<li>FOLLOWER: 127.0.0.2 |

##### What is the `hash_split` value in the Partition column? 

For a table with hash sharding, each tablet has a Partition value that shows a  `hash_split` . The `hash_split` details a range of hexadecimal values. 
```
hash_split: [0x0000, 0x5555)
hash_split: [0x5555, 0xAAAA)
hash_split: [0xAAAA, 0xFFFF)
```
The conversion of the above hexadecimal values to integer values is as follows:
```
0-21845
21845-43690
43690-65535
```

`tbl_pk_hash` uses consistent has sharding with the partition key being `k`.  All tablets will store rows using the hash coded values for `k`.

### q2a | yb_hash_code()
YSQL has an built-in function that shows the integer form of a hashed value, `yb_hash_code()`, in integer form. Using this function, you can view the hash code value of the primary key, `k`, for `tbl_pk_hash`. 

The resulting integer value places the row in the the hash_split range of the tablets.

In the web ui, you can then confirm that the PK has a hash code that lives in the hash_split range of the tablet leader in question.  To help make the conversion from hexadecimal to integer easier, the following query use various utility user-defined functions and shows:
- k, the primary key value
- the yb_hash_code() integer for k
- and hash_split hexadecimal range in integer form

In [ ]:
%%sql /* query with hash buckets */
select '' as _
  , k
 -- , v
  , yb_hash_code(k::int) as k_hash_code
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0x0000, 0x5555)'::text) as col_0x0000_0x5555
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0x5555, 0xAAAA)'::text) as col_0x5555_0xAAAA
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0xAAAA, 0xFFFF)'::text) as col_0xAAAA_0xFFFF
from tbl_pk_hash
where 1=1 
-- and k=123
order by k asc
-- order by k desc
-- offset 330
limit 30
;


### q2 | Obsevations
The results show that the 30 returned rows are distributed among all of the hash_split ranges, meaning that each tablet leader is returning results. The results are sorted in an ascending order of K values.

#### q2b | Experiment
Question: 
- In the query above, comment out the `order by k` line. Rerun the query cell. What observations can you make about the results?

Answer:
- The results are in the order of `k_hash_code`, not `k`.
- YugabyteDB only returns `k_hash_code` ordered results and these 30 row sare from the first hash_split range, `0-21845`.
- The `k_hash_code` ordering is how the internal LSM-Tree sorted the table inserts.
- For the given tablet, the LSM-tree persists to disk as Sorted Sequence Tables (SST files).
- The reads come from the SST file for the tablet. For this reason, you can rerun the query over and over again and still see the same "order" (`k_hash_code`) of "unordered" (`k`) results.


Question: 
- In the query above, with the `order by k` line already commented out, uncomment the `offset 330` line. Rerun the query cell. What observations can you make about the results?

Answer:
- YugabyteDB only returns `k_hash_code` ordered results from the beginning of the offset, which spans two hash_split ranges, indicating that the hash sharding is more or less evenly split among the tablets.


### q2c | View the explain plan and metrics
This is the same exact query above. This time, however, you will generate an Explain Plan and the view the metrics.

Just like the section before, you wil rerun the queries to answer the experiment questions.

In [ ]:
%%sql /* explain plan */
execute stmt_util_metrics_snap_reset;
explain (costs off, analyze, verbose) 
select '' as _
  , k
 -- , v
  , yb_hash_code(k::int) as k_hash_code
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0x0000, 0x5555)'::text) as col_0x0000_0x5555
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0x5555, 0xAAAA)'::text) as col_0x5555_0xAAAA
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( k::int),'hash_split: [0xAAAA, 0xFFFF)'::text) as col_0xAAAA_0xFFFF
from tbl_pk_hash
where 1=1 
-- and k=123
order by k asc
-- order by k desc
offset 330
limit 30
;

##### q2c | Explain Plan (above ^^)
For the initial query with the `order by asc` clause, the Explain Plan shows that this is a `Seq Scan` query, and no index is being used:

`-> Seq Scan on public.tbl_pk_hash (actual time=2.626..7.200 rows=1000 loops=1)`

A `Seq Scan` results in potentially accessing all the rows in the table.  These means all tablet leaders will most likely process the query.

In [ ]:
%%sql
execute stmt_util_metrics_snap_table;

#### q2c | Metrics (above ^^)
In the initial query, the `Seq Scan` accesses all tablets leaders:

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_1	 | 1 | 	689 |
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_2	| 1	| 641 |
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_3	| 1	| 667 |

#### q2c | Experiment
Question: 
- When you change the SQL code above and comment out the `ORDER BY` clause and rerun the query, what observations can you make about the Explain Plan and the Metrics.

Answer:
- Even though the Explain Plan shows that this is a `Seq Scan` query:
  - `-> Seq Scan on public.tbl_students (actual time=1.072..5.176 rows=30 loops=1`

The table scan is only for the tablet that contains the hash_split range where 30 rows live.
| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_1	 | 1 | 	62 |

- YugabyteDB returns the unordered results and from the first hash_split range, `0-21845`.
- The "unordered" results are actually returned as they were ordered in the LSM-Tree as Sorted Sequence Tables (SST files). For this reason, you can rerun the query over and over again and still see the same "order" of "unordered" results. 


Question: 
- With the `ORDER BY` clauses commented out, what happens when you uncomment one of the `OFFSET` lines and run the query again? How does the Explain Plan compare to the Metrics?


Answer:
- YugabyteDB returns results from the beginning of the offset, which, if the hash sharding is more or less evenly split, should span two hash_split ranges. Even though the Explain Plan shows that this is a `Seq Scan` query:
  - `-> Seq Scan on public.tbl_students (actual time=4.039..55.580 rows=360 loops=1)`

The table scan is for the tablets that contains the hash_split ranges. This means full table scans for two tablet leaders.

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_1	 | 1 | 	689 |
| db_ybu tbl_pk_hash link_table_id tablet_id_unq_2	| 1	| 641 |


---
## Range Sharding
Range sharding splits the rows of a table into contiguous ranges that respect the sort order of the table based on the primary key column values. Typically, a range sharded table begins with only one tablet. With more row insertions, the single tablet eventually splits into two or more tablets. As the data volume for the range sharded table continues to grow, so does the number of tablets for the table. YugabyteDB balances the number of tablet leaders across the YB-TServers in the cluster. Here is an illustrations of range sharding:

<img  style="background-color:#FFF" src="assets/range_sharding.png" >

 Queries that specify a range of values in the query predicate often benefit from range sharding.

## q3 | Create a PK table with range sharding
The following table will take about
- Query when 1000 rows --> results, see natural order
- Query when 1000 rows --> results, query in reverse order with order by desc
- Query with predicate, offset, limit
- Compare hash vs range for range predicate
  


In [ ]:
%%sql  /* create table no PK and insert rows */
drop table if exists tbl_pk_range;

create table if not exists tbl_pk_range (k int, v text, primary key (k asc)) ;

insert into tbl_pk_range (k,v)
select g.id, format('%s%s',chr(97+CAST(random() * 25 AS INTEGER)),chr(97+CAST(random() * 25 AS INTEGER)))
from generate_series(1, 1000) AS g (id);

#### View in the YB-Master web ui for the table details
Open the Web UI for the table. Copy and paste the following URL in your web browser:

In [9]:
#%% python, but prepared statements as sql magic
WORKSPACE_URL = MY_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
view_url = %sql select fn_get_table_id(:WORKSPACE_URL,8200,'tbl_pk_range') as open_this_url_in_your_web_browser ;

print (view_url)

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.
+--------------------------------------------------------------------------------------------------------------+
|                                      open_this_url_in_your_web_browser                                       |
+--------------------------------------------------------------------------------------------------------------+
| https://7000-yugabytedbuni-ybugitpod-sgjfjfecua0.ws-us53.gitpod.io/table?id=00004000000030008000000000004070 |
+--------------------------------------------------------------------------------------------------------------+


#### Review the Column section
In the column section for `tbl_pk_range`, you will see:


| Column | ID	| Type |
|--------|------|------|
| k	         | 1	| int32 NOT NULL NOT A PARTITION KEY | 
| v	         | 2	| string NULLABLE NOT A PARTITION KEY | 

<br/>

> Note
>  
> There is no PARTITION KEY (shard key) for the `tbl_pk_range` table. The table specifies a Range Sharding strategy for the primary key (PK).  `k`, the PK, is, however, shown as `NOT NULL`.


#### Review the Tablet section
The Tablet section shows the details for the existing tablets. Here is an abridged example of the Tablet section for the `tbl_pk_hash` table:

| Tablet ID |	Partition	| SplitDepth	| State	| Hidden	| Message	| RaftConfig|
|--|--|--|--|--|--|--|
| some_uuid_1<br>`42ca468817c145c29145752718b7c6a7` |	`range: [<start>, <end>)` |	0	| Running|	false| Tablet reported with an active leader	|<li>FOLLOWER: 127.0.0.1 <li>FOLLOWER: 127.0.0.3<li>LEADER: 127.0.0.2  |

YugabyteDB will automatically split this tablet based on the size of the table on disk. The following global flags determine this behavior:

```
--tablet_force_split_threshold_bytes=107374182400 --> 10240 MB
--tablet_split_high_phase_shard_count_per_node=24
--tablet_split_high_phase_size_threshold_bytes=10737418240 --> 10240 MB
--tablet_split_low_phase_shard_count_per_node=8
--tablet_split_low_phase_size_threshold_bytes=536870912 --> 512 MB
--tablet_split_size_threshold_bytes=0
```

The low phase accounts for when initial tablet splits are relatively low. With more data volume, the threshold increases from 512 MB to over 10 GBs.

#### q3a | Natural order
Unlike the consistent hash sharding of `tbl_pk_hash`, `tbl_pk_range` preserves the order of the PK. The following query shows how:

In [17]:
%%sql /* explain plan */
select '' as _
  , k
from tbl_pk_range
where 1=1 
and k = 123
-- and k between 100 and 200
-- order by k desc
limit 30
;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
30 rows affected.


k
1
2
3
4
5
6
7
8
9
10


The above result reveals that that the range sharding stores data in the specified order of the primary key definition, which by default is `asc`, ascending order.

In [23]:
%%sql /* explain plan */
execute stmt_util_metrics_snap_reset;
explain (costs off, analyze, verbose) 
select '' as _
  , k
from tbl_pk_range
where 1=1 
and k = 123
-- and k between 100 and 200
-- order by k desc
limit 30
;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
0 rows affected.
6 rows affected.


QUERY PLAN
Index Scan Backward using tbl_pk_range_pkey on public.tbl_pk_range (actual time=3.492..21.868 rows=999 loops=1)
"Output: ''::text, k"
Index Cond: ((tbl_pk_range.k >= 1) AND (tbl_pk_range.k <= 999))
Planning Time: 0.088 ms
Execution Time: 21.968 ms
Peak Memory Usage: 8 kB


#### q3a | Explain Plan (above ^^)

The Explain Plan shows that this query uses the PK index and reads just one row..
- `-> Index Scan using tbl_pk_range_pkey on public.tbl_pk_range (actual time=7.473..7.475 rows=1 loops=1)`

The Index Condition reflects the query predicate.

- `Index Cond: (tbl_pk_range.k = 123)`

To view the DocDB metrics, run the following prepared statement:

In [24]:
%%sql
execute stmt_util_metrics_snap_table;

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
1 rows affected.


[dbname | relname | tableid | tabletid | isLeader],rocksdb_number_db_seek,rocksdb_number_db_next,rows_inserted
db_ybu | tbl_pk_range | https://7000-yugabytedbuni-ybugitpod-sgjfjfecua0.ws-us53.gitpod.io/table?id=00004000000030008000000000004070 | 42ca468817c145c29145752718b7c6a7 | L,2058,2028,None


#### q3a | Metrics (above ^^)

The Explain Plan revealed that the query uses the PK index and reads only one row. Because there is only one tablet at this time for the range sharded table, the Metrics reports shows that the query accesses a single tablet. There is on seek for the offset and two reads.

#### q3a | Experiment
Question: 
- Change the SQL code above the Explain plan. Comment out the `and k = 123` line and uncomment the `and k between 100 and 200` line. Rerun the query to review the Explain Plan. Rerun the Metrics prepared statement. What observations can you make about the Explain Plan and the Metrics output?

Answer:
- The query uses the PK index. The only difference in metrics is the number of reads (db_next), which reflects the limit on the number of rows.
- 

Question: 
- Uncomment the `order by k desc` line. Rerun the Explain Plan and the Metrics prepared statement. What new observations can you make? 


Answer:
- The Explain Plan shows that query uses `-> Index Scan Backward` for the PK index.
- The Metrics shows that the Index Scan Backward operations resulted in 64 seeks and only 62 reads. For thirty rows (`limit 30`), this means that there a two seeks per each backwards operation!


| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_pk_range link_table_id tablet_id_unq_1	 | 64 | 	62 |


When fetching rows in reverse order of the index, with `Index Scan Backward`, there is no next call to go backwards. Instead, there are two seeks to go backwards. For this reason, the range sharding order of the primary key (`asc` or `desc`) may not be optimal for queries with an `Index Scan Backward` plan. 


---
## Summary
In this notebook, you learned about how YugabyteDB uses two distinct sharding strategies for storing data in a distributed manner:
- consistent hash sharding
- range sharding

You specify the sharding strategy in the Primary Key definition of a table: `hash` for hash sharding, and either `asc` or `desc` for range sharding.
By default, YSQL for YugabyteDB employs consistent hash sharding, even if a primary key is not defined.

### Consistent hash sharding
With consistent hash sharding, YugabyteDB evenly and randomly distributes data across tablets using a partitioning algorithm. By computing a consistent hash on the partition column values of that row, YugabyteDB places each row of the table into a tablet. By using the `yb_hash_code()` built-in function, you can view the integer form of a hash coded value.

#### Benefits of hash sharding
Because consistent hash sharding distributes data evenly across all the nodes in the cluster while also retaining the ability to horizontally scale (add or remove nodes) in a YugabyteDB cluster, the strategy is ideal for massively scalable workloads that require shard key predicate queries.

#### Concerns with hash sharding
However, performing range queries could be inefficient. Examples of range queries are finding rows greater than a lower bound or less than an upper bound (as opposed to point lookups).


### Range sharding
Range sharding involves splitting the rows of a table into contiguous ranges that respect the sort order of the table based on the primary key column values. YugabyteDB initially creates a single tablet leader for a table. with range sharding. As the data volume increases for the table, YugabyteDB dynamically splits the initial tablet into two or moore tablets. 

#### Benefits of range sharding
Range harding allows efficiently querying a range of rows by the primary key values. Examples of such a query is to look up all keys that lie between a lower bound and an upper bound.


#### Concerns with range sharding
Range sharding can introduce a number of issues in practice and at scale. Starting out with a single tablet leader implies only a single node is handling all the user queries, making this node in the cluster "hot" in terms of CPU, Memory,  Network, and Disk utilization. To alleviate this side effect, the data volume must grow significantly in order for the tablet to split. Another concern is that the global ordering of keys across all the tablets itself generates hot spots. As a result, certain tablet will naturally have more activity than others, resulting in an node with increased utilization of CPU, Memory, Network, and Disk.

## Additional reading

To learn more about tables, tablets, and sharding, check out these resources in the Course Player.